In [1]:
import pandas as pd
from  work_fafcd0abeb574e558b3b719fe5acadb1 import TabularDataSet
import numpy as np
import pandas as pd
import torch
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import torch.nn as nn
from collections import Counter
from sklearn.preprocessing import LabelEncoder

ModuleNotFoundError: No module named 'work_fafcd0abeb574e558b3b719fe5acadb1'

In [2]:
data = pd.read_csv('./work_fafcd0abeb574e558b3b719fe5acadb1/pytoch-train.csv', encoding="GBK")

FileNotFoundError: [Errno 2] No such file or directory: './work_fafcd0abeb574e558b3b719fe5acadb1/pytoch-train.csv'

In [3]:
for d in data.columns:
    if data[d].dtype == "object":
        data.drop(d, axis=1, inplace=True)

NameError: name 'data' is not defined

In [ ]:
data.fillna(0, inplace = True)

In [ ]:
Y = data['yc']
Y_label = LabelEncoder()
Y = Y_label.fit_transform(Y)
X = data.drop(columns="yc")

In [ ]:
train_ds = TabularDataSet.tabularDataset(X, Y)

# 训练的过程中，使用DataLoader加载数据
train_loader = DataLoader(train_ds, batch_size=64, shuffle=True)

In [ ]:
# 卷积(Conv2d) -> 激励函数(ReLU) -> 池化, 向下采样 (MaxPooling) ->
# 再来一遍 -> 展平多维的卷积成的特征图 -> 接入全连接层 (Linear) -> 输出

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential( # 1x28x28
            nn.Conv2d(
                in_channels=1,
                out_channels=16,
                kernel_size=5,
                stride=1,
                padding=2,
            ),# 16x28x28
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.conv2 = nn.Sequential( # 16x14x14
            nn.Conv2d(16, 32, 5, 1, 2),# 32x14x14
            nn.ReLU(),
            nn.MaxPool2d(2),# 32x7x7
        )
        self.out = nn.Linear(32*7*7, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1)  # 展平多维的卷积图成 (batch_size, 32 * 7 * 7)\
        output = self.out(x)
        return output

In [ ]:

import torch
import torch.nn as nn
import torch.utils.data as Data
import  torchvision
import matplotlib.pyplot as plt

torch.manual_seed(1)

EPOCH = 1
BATCH_SIZE = 50
LR = 0.001
DOWNLOAD_MNIST = True

cnn = CNN()

#将模型放入GPU中
cnn = cnn.cuda()
print(cnn)

optimizer = torch.optim.Adam(cnn.parameters(), lr=LR)
loss_func = nn.CrossEntropyLoss()

for epoch in range(EPOCH):
    for step, (b_x, b_y) in enumerate(train_loader):
        output = cnn(b_x.cuda())
        loss = loss_func(output, b_y.cuda())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

#test_output = cnn(test_x[:10])
#pred_y = torch.max(test_output, 1)[1].data.cpu().numpy().squeeze()
#print(pred_y, 'prediction number')
#print(test_y[:10].cpu().numpy(), 'real number')

In [ ]:
!cat /usr/local/cuda/version.txt

In [ ]:
!cat /proc/driver/nvidia/version